<a href="https://colab.research.google.com/github/RicNavarro/personal-projects/blob/main/DC_Rebirth_List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [297]:
import numpy as np
import pandas as pd
import glob as gb
import os
import re

Some additional cleaning was done in Excel
1) Instances of "," within certain entries were removed
2) Some text was properly re-encoded (entries with single quotes, for example, were faulty)
3) A few entries contained not the names of the issues, but descriptions of how to read certain events. Those were removed

In [298]:
df_main = pd.read_csv("data/DC_Rebirth_Comic_Order.csv")

In [299]:
df_main.head()

,Comic Name
0,DC Universe: Rebirth #1 (2016)
1,The Flash: Rebirth Vol. 2 #1 (2016)
2,Nightwing: Rebirth #1 (2016)
3,Titans: Rebirth #1 (2016)
4,Titans Vol. 3 #1 (2016)


We proceed to do some initial formatting in order to extract the character names, whilst removing spacing, parenthesis, issue nnumbering and some other info that won't be relevant for the "Character" field. Also, we distinguish the entrie that indicate an event

In [357]:
def clean_name(name):
    if name.split(" ")[0] == 'Read':
        return 'EVENT'
    else:
        name = name.title()
        if ":" in name:
            name = name.split(':')
        
        # Some Comic Names have the character name after the colon when they're part of an event
            split_to_keep = 0
            
            if name [0] == "" or "BATMAN" in name[1].upper():    
            # The "BATMAN" exception is added specifically because of a single edge case where the title is
            # not part of an event, but the character's name is after the colon
                split_to_keep = 1
            name = name[split_to_keep]

        return (name.split('Vol')[0].split('#')[0]).split("Annual")[0].split("Special")[0].rstrip().lstrip().upper()

df_main['Character'] = df_main['Comic Name'].apply(clean_name)

In [358]:
df_main

,Comic Name,Character,Event
0,DC Universe: Rebirth #1 (2016),DC UNIVERSE,None
1,The Flash: Rebirth Vol. 2 #1 (2016),THE FLASH,None
2,Nightwing: Rebirth #1 (2016),NIGHTWING,None
3,Titans: Rebirth #1 (2016),TITANS,None
4,Titans Vol. 3 #1 (2016),TITANS,None
...,...,...,...
2703,Future State: Superman: House of El #1 (2021),FUTURE STATE,None
2704,Future State: Swamp Thing #1 (2021),FUTURE STATE,None
2705,Future State: Swamp Thing #2,FUTURE STATE,None
2706,Future State: Immortal Wonder Woman #1 (2021),FUTURE STATE,None


## Before stripping the 

**Eventização**

O código abaixo deve ser retrabalhado para que todas as entradas onde há 'Character' igual a 'EVENT' sejam stripadas de 'Read ' e ' here.'

In [359]:
def clean_event(x):
  return x.replace('Read ', '').replace(' here.', '')

events_true = df_main['Character'] == "EVENT"
print(events_true)

df_main['event_names_clean'] = df_main.loc[events_true, 'Comic Name'].apply(clean_event)

0       False
1       False
2       False
3       False
4       False
        ...  
2703    False
2704    False
2705    False
2706    False
2707    False
Name: Character, Length: 2708, dtype: bool


In [360]:
df_main['Event'] = np.where(df_main['Character'] == "EVENT", df_main['event_names_clean'], "None")
df_main = df_main.drop("event_names_clean", axis="columns")
df_main['Event']

0       None
1       None
2       None
3       None
4       None
        ... 
2703    None
2704    None
2705    None
2706    None
2707    None
Name: Event, Length: 2708, dtype: object

In [361]:
df_main

,Comic Name,Character,Event
0,DC Universe: Rebirth #1 (2016),DC UNIVERSE,None
1,The Flash: Rebirth Vol. 2 #1 (2016),THE FLASH,None
2,Nightwing: Rebirth #1 (2016),NIGHTWING,None
3,Titans: Rebirth #1 (2016),TITANS,None
4,Titans Vol. 3 #1 (2016),TITANS,None
...,...,...,...
2703,Future State: Superman: House of El #1 (2021),FUTURE STATE,None
2704,Future State: Swamp Thing #1 (2021),FUTURE STATE,None
2705,Future State: Swamp Thing #2,FUTURE STATE,None
2706,Future State: Immortal Wonder Woman #1 (2021),FUTURE STATE,None


Vamos ter de usar expressões regulares para poder modificar adequadamente as entradas na Série `events_list`, removendo espaços e pontuações para que nossa comparação com o nome dos arquivos seja adequada.

Consta no código uma explicação sobre as expressões

In [362]:
events_list_entry_names = events_list = df_main.loc[df_main['Event'] != 'None']['Event']
print (events_list_entry_names)

def reformat_name_event(event):
    # Substituir qualquer caractere n alfanum (exceto _) ou espaço por um underscore
    #    Regex: [^\w\s] -> qualquer coisa que NÃO seja (caractere alfanumérico OU espaço)
    #    | -> OU
    #    \s+ -> um ou mais espaços
    event = re.sub(r'[^\w\s]|\s+', '_', event)

    # Remover underscores duplicados que possam surgir (ex: "Night__of___the")
    event = re.sub(r'__+', '_', event)

    # Remover underscores no início ou fim da string
    event = event.strip('_')

    return event


events_list_file_names = events_list.apply(reformat_name_event)
print (events_list_file_names)

Series([], Name: Event, dtype: object)
Series([], Name: Event, dtype: object)


Temos que ler os nomes dos arquivos em DC_Events e

**(A)** Conforme fazemos a leitura armazenamos na lista `df_events`, que depois é reordenada segundo `events_dict` ou similares

ou

**(B)** Fazemos a leitura já checando os elementos de `events_dict`, ordenando conforme lemos e armazenando em `dfs_events` já na ordem

In [363]:
col_to_keep = "Comic Name"

events_path = "data/DC_Events/"
fileformat = "*.csv"
event_names = []
dfs_events = []

path_events = gb.glob(f"{events_path}/{fileformat}")

#for i in events_list.values:
#  print(i)
#print ("File names\n")
#for i in path_events:
#  print(os.path.basename(i))

#print ("\nEvent Column names\n")
if (len(path_events) == len(events_list_file_names.values)):
  for i in range(len(path_events)):
    #print(events_path + events_list.values[i] + fileformat.strip("*"))
    try:
        # remeber: the events are added according to the order they were read from in the original Comic_Order file
        event_address = events_path + events_list_file_names.values[i] + fileformat.strip("*")
        df_toshorten = pd.read_csv(event_address)
        dfs_events.append(df_toshorten[[col_to_keep]])
    except:
      print("The " + events_list_file_names.values[i] + " event has no corresponding file")

dfs_events

[]

### Getting dataframes from each additional file and adding a column with the proper event name

In [364]:
def clean_event_name(comic_name, event):
    if not(event in comic_name):
       return clean_name(comic_name) 
        
    replaced_name = clean_name(comic_name.replace(event, ""))
    if len(replaced_name) != 0:
        return replaced_name
    else:
        return event.upper()
    
for i in range(len(events_list_entry_names)):

    dfs_events[i]["Event"] = events_list_entry_names.iloc[i]
    
    # Events require a special cleanup, considering many of them have the Event name in the Comic Book's title,
    # which would make some noise when doing the regular cleanup

    dfs_events[i]['Character'] = dfs_events[i].apply(lambda x: clean_event_name(x['Comic Name'], x['Event']), axis=1)

#Checking one of the dfs for the appropriate column
dfs_events

[]

Next step is to do the dataframe merging

In [365]:
df_main.loc[df_main['Event'] == "Justice League vs. Suicide Squad"]

,Comic Name,Character,Event


In [366]:
insertion_points = []
#for i in dfs_events:
    
for i in df_main.loc[df_main['Event'] != 'None']['Comic Name']:
    insertion_points.append(i)

#print(insertion_points)

for i in range(len(insertion_points)):
    insertion_index = df_main[df_main['Comic Name'] == insertion_points[i]].index[0]

    df_main_slice1 = df_main.iloc[:insertion_index]
    df_main_slice2 = df_main.iloc[insertion_index + 1:]

    df_main = pd.concat([df_main_slice1, dfs_events[i], df_main_slice2], ignore_index=True)

In [367]:
df_main

,Comic Name,Character,Event
0,DC Universe: Rebirth #1 (2016),DC UNIVERSE,None
1,The Flash: Rebirth Vol. 2 #1 (2016),THE FLASH,None
2,Nightwing: Rebirth #1 (2016),NIGHTWING,None
3,Titans: Rebirth #1 (2016),TITANS,None
4,Titans Vol. 3 #1 (2016),TITANS,None
...,...,...,...
2703,Future State: Superman: House of El #1 (2021),FUTURE STATE,None
2704,Future State: Swamp Thing #1 (2021),FUTURE STATE,None
2705,Future State: Swamp Thing #2,FUTURE STATE,None
2706,Future State: Immortal Wonder Woman #1 (2021),FUTURE STATE,None


-----
## Removal of character name redundancies (WIP)

In [368]:
all_heroes = df_main['Character']
for i in all_heroes:
    if "JOKER" in i:
        print (i)

# Joker special

#for i in all_heroes:
#    if "JOKER" in i:

df_main.loc[df_main['Character'] == "THE JOKER"]

THE JOKER
THE JOKER 80TH ANNIVERSARY 100-PAGE SUPER SPECTACULAR


,Comic Name,Character,Event
2027,The Joker: Year of the Villain #1 (2019),THE JOKER,None


In [369]:
print("There are " + str(len(df_main['Character'].unique())) + " characters")
unique_heroes = df_main['Character'].unique()
unique_heroes

There are 151 characters


array(['DC UNIVERSE', 'THE FLASH', 'NIGHTWING', 'TITANS', 'SUPERMAN',
       'ACTION COMICS', 'SUPERWOMAN', 'BATMAN', 'GREEN ARROW',
       'SUICIDE SQUAD', 'HAL JORDAN & THE GREEN LANTERN CORPS',
       'GREEN LANTERNS', 'HARLEY QUINN', 'AQUAMAN', 'BATGIRL',
       'BATGIRL AND THE BIRDS OF PREY', 'DETECTIVE COMICS', 'TEEN TITANS',
       'JUSTICE LEAGUE', 'TRINITY', 'CATWOMAN', 'CYBORG',
       'GOTHAM ACADEMY', 'WONDER WOMAN', 'DC REBIRTH HOLIDAY',
       'NEW SUPER-MAN', 'DEATHSTROKE', 'RAVEN', 'ALL-STAR BATMAN',
       'RED HOOD AND THE OUTLAWS', 'JUSTICE LEAGUE VS. SUICIDE SQUAD',
       'JUSTICE LEAGUE OF AMERICA', 'SUPER SONS', 'THE HELLBLAZER',
       'SIXPACK AND DOGWELDER', 'DEADMAN', 'SUPERGIRL', 'MOTHER PANIC',
       'BATWOMAN', 'BATMAN BEYOND', 'BLUE BEETLE',
       'CAVE CARSON HAS A CYBERNETIC EYE', 'DEATH OF HAWKMAN',
       'MIDNIGHTER AND APOLLO', 'SHADE THE CHANGING GIRL', 'BANE',
       'DOOM PATROL', 'THE ODYSSEY OF THE AMAZONS',
       'THE FALL AND RISE OF CAPT

### An evaluation of what constitutes a **"collab title"** is needed
In order to do that, we look for some patterns within the `Character` column. The ones identified were the usage of "AND", "VS." and "/" to signal the involvement of at least two characters. (This is not perfect, as we'll see an edge case bellow)

In [370]:
for i in unique_heroes:
    if "AND" in i or "VS" in i or "/" in i or "\\" in i:
        print(i)

#print("--------------------------------------")
#for i in unique_heroes:
#    if "OF" in i:
#        print(i)

BATGIRL AND THE BIRDS OF PREY
RED HOOD AND THE OUTLAWS
JUSTICE LEAGUE VS. SUICIDE SQUAD
SIXPACK AND DOGWELDER
MIDNIGHTER AND APOLLO
THE FALL AND RISE OF CAPTAIN ATOM
HAL JORDAN AND THE GREEN LANTERN CORPS
JLA/DOOM PATROL
MOTHER PANIC/BATMAN
SHADE THE CHANGING GIRL/WONDER WOMAN
CAVE CARSON HAS A CYBERNETIC EYE/SWAMP THING
DOOM PATROL/JLA
BATMAN AND THE SIGNAL
NEW SUPER-MAN AND THE JUSTICE LEAGUE OF CHINA
BATMAN AND WONDER WOMAN
WONDER WOMAN AND DARK JUSTICE LEAGUE
DARK JUSTICE LEAGUE AND WONDER WOMAN
JUSTICE LEAGUE/AQUAMAN
AQUAMAN/JUSTICE LEAGUE
BATMAN AND THE OUTSIDERS
BATMAN/SUPERMAN
BATMAN VS. RA'S AL GHUL
HARLEY QUINN AND THE BIRDS OF PREY


That looks good, except in a single instance: `THE FALL AND RISE OF CAPTAIN ATOM`. This title will have to be dealt with separately
Other than that, we're good to use those parameters to create a Side Character column